In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('mixtral-GAD')

In [ ]:
import requests
import json

input_file_path = "sample_data/some-100.json"
output_file_path = "sample_data/mixtral-gad-eval.json"

# input_file_path = "sample_data/test_original.json"
# output_file_path = "sample_data/test_output.json"

In [ ]:
with open(input_file_path, "r") as input_file:
    input_data = json.load(input_file)

In [ ]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

base_instruction = """ When given the below JSON formatted file content, I need you to give me the specific sentences from the text that exhibit a set of symptoms which are under the annotations section. Below is an example of INPUT and OUTPUT. Just give me the outputs, do not give any code or re-output the prompting, do not just give "yes" or "no" again, give the actual sentences. Keep JSON Formatting. And make sure to always return all the components of each of the objects, so always include "post_text", "post_title", and "annotations":

Input:

{
{
"post_title": "How do I let go of my anxiety over dying?", "post_text": "It is all I can think about for the last 3 years. During this time, I have shut out my family and cut off all of my friends because I don’t see the point in making meaningful connections. I have lost interest in everything I used to enjoy because I’m too scared to leave my house. It’s affecting my quality of life and I’m starting to think of killing myself to feel like I have some control. Has anyone felt like this and if so, have you and how did you overcome it?",
"annotations": [
"Feeling-nervous-anxious-or-on-edge": [ ]
, "Not-being-able-to-stop-or-control-worrying": [ ]
, "Worrying-too-much-about-different-things": [ ]
, "Trouble-relaxing": [ ]
, "Being-so-restless-that-it-is-hard-to-sit-still": []
, "Becoming-easily-annoyed-or-irritable": []
, "Feeling-afraid-as-if-something-awful-might-happen":[ ]
]
}
}

And here is the example Output:
{
    "post_title": "How do I let go of my anxiety over dying?",
    "post_text": "It is all I can think about for the last 3 years. During this time, I have shut out my family and cut off all of my friends because I don’t see the point in making meaningful connections. I have lost interest in everything I used to enjoy because I’m too scared to leave my house. It’s affecting my quality of life and I’m starting to think of killing myself to feel like I have some control. Has anyone felt like this and if so, have you and how did you overcome it?",
    "annotations": {
        "Feeling-nervous-anxious-or-on-edge": [
            "It is all I can think about for the last 3 years.",
            "I’m too scared to leave my house."
        ],
        "Not-being-able-to-stop-or-control-worrying": [
            "It is all I can think about for the last 3 years."
        ],
        "Worrying-too-much-about-different-things": [],
        "Trouble-relaxing": [
            "I’m too scared to leave my house."
        ],
        "Being-so-restless-that-it-is-hard-to-sit-still": [],
        "Becoming-easily-annoyed-or-irritable": [],
        "Feeling-afraid-as-if-something-awful-might-happen": [
            "It’s affecting my quality of life and I’m starting to think of killing myself to feel like I have some control."
        ]
    }
}


May I proceed with the rest of the INPUTS? """

In [ ]:
import json

results = []

# Function to process each JSON object and collect the output
def process_and_collect_output(obj):
    # Construct the prompt
    full_prompt = f"{base_instruction} {json.dumps(obj, indent=2)}"

    # Prepare the payload
    payload = {
        "messages": [
            {"role": "user", "content": full_prompt}
        ],
        "model": "mixtral-8x7b-32768"
    }

    try:
        # Call the Groq API
        chat_completion = client.chat.completions.create(**payload)

        # Extract the LLM output
        llm_output = chat_completion.choices[0].message.content

        # Clean up the LLM output (remove unwanted characters or text if necessary)
        llm_output = llm_output.strip()  # Strip any extra whitespace
        # Handle cases where output is not valid JSON
        try:
            result = json.loads(llm_output)
        except json.JSONDecodeError:
            print(f"Error decoding JSON from output: {llm_output}")
            result = {}  # Use an empty dictionary or handle as needed

        # Append output to results list
        results.append(result)

    except Exception as e:
        print(f"Error processing object: {e}")

# Process and collect results for each object
for obj in input_data:
    process_and_collect_output(obj)

# Write results to output file with comma-separated JSON objects
with open(output_file_path, "w") as output_file:
    output_file.write("[\n")  # Start the JSON array
    first = True
    for result in results:
        if not first:
            output_file.write(",\n")
        json.dump(result, output_file, indent=2)
        first = False
    output_file.write("\n]")  # End the JSON array

Error decoding JSON from output: Here are the specific sentences from the text that exhibit the set of symptoms under the annotations section:

Input:

{
"post\_title": "I don't feel original anymore.",
"post\_text": "When I was in high school a few years back, I was one of the highest competitors in my school. I joined the high school band in freshman year and by senior year I became one of the best in my section. My academics were always straight and I exercised daily. Senior year I enlisted in the military and now I believe it was one of my worst decisions in life. Before I went to boot camp I was motivated, a patriot and believed that the elite joined the military. In senior year I never applied for any scholarships and I was offered one but turned it down because I already signed the papers. I thought I set myself up for success. Now I believe I was dead wrong for joining. The only benefit I see so far after a year and a half of service is that I'm trying to set myself up financia